### Setting with Colab

In [ ]:
# 구글드라이브 연동

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 모듈 import

# !pip install tensorflow-addons
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import  keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
from keras.models import load_model
from PIL import Image
import math
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tqdm import tqdm
%matplotlib inline

In [ ]:
# set working directory

os.chdir('/content/drive/MyDrive/Colab Notebooks/데이콘_mnist')
#!pwd 

### Data Preparing

In [ ]:
# 생성한 디렉토리에 zip파일 압축해제

# shutil.rmtree("dacon_m12/dirty_mnist3", ignore_errors=True)  # 빈 폴더가 아니더라도 무시하고 삭제하는 코드
!mkdir "dacon_m12/dirty_mnist3"
!unzip "dacon_m12/dirty_mnist.zip" -d "dacon_m12/dirty_mnist3"

In [ ]:
# 이미지 데이터 확인하기

image = Image.open("hj/dirty_mnist/00001.png")
image

In [ ]:
# Generator를 사용하기 위해 dataframe 만들기

df = pd.read_csv('dacon_m12/dirty_mnist_answer.csv')
df['index'] = df['index'].apply(lambda x: str("{:0>5d}".format(x))+'.png')

### Data preprocessing

In [ ]:
columns = list(map(chr, range(97, 123))) # 'abcde...xyz'를 만들어주는 코드

# Data augmentation
datagen=ImageDataGenerator(rescale=1./255., validation_split=0.1,
                           rotation_range=30,
                           horizontal_flip=True,
                           vertical_flip=True)

# Generator (train & test)
train_gen = datagen.flow_from_dataframe(dataframe=df,
                                        directory=train_data_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size=32,
                                        seed=1,
                                        color_mode='rgb',
                                        shuffle=True,
                                        class_mode='raw',
                                        target_size=(128, 128),
                                        subset='training')

val_gen = datagen.flow_from_dataframe(dataframe=df,
                                        directory=train_data_dir,
                                        x_col='index',
                                        y_col=columns,
                                        batch_size=32,
                                        seed=1,
                                        color_mode='rgb',
                                        shuffle=True,
                                        class_mode='raw',
                                        target_size=(128, 128),
                                        subset='validation')

In [ ]:
# 데이터 확인하기 1

x_train, y_train = train_gen.next()
for idx in range(3):  
    print(x_train[idx].shape)
    print(y_train[idx])

In [ ]:
# 데이터 확인하기 2 - 이미지 확인 (Generator에서 batch_size=1 지정 필수!)

X, y = train_gen.next()
X_reshape = X.reshape(256, 256)

plt.figure(figsize=(5, 5)) 
plt.axis('off')
plt.imshow(X_reshape, cmap='gray')
plt.show()

### Modeling
#### 1. (Pretrained) InceptionResnet + Lookahead 
- 30 epochs
- acc : 89.98%

In [ ]:
tf.random.set_seed(1)
InceptionResNetV2_Lookahead = tf.keras.Sequential([tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3)),
                              tf.keras.layers.GlobalAveragePooling2D(),
                              tf.keras.layers.Dense(1024, kernel_initializer='he_normal'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Activation('relu'),
                              tf.keras.layers.Dense(512, kernel_initializer='he_normal'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Activation('relu'),
                              tf.keras.layers.Dense(256, kernel_initializer='he_normal'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Activation('relu'),
                              tf.keras.layers.Dense(26, kernel_initializer='he_normal', activation='sigmoid', name='predictions')
                              ])

In [ ]:
# 모델 구조 시각화
SVG(model_to_dot(InceptionResNetV2_Lookahead, show_shapes=True, dpi=65).create(prog='dot', format='svg'))

In [ ]:
nadam = tf.keras.optimizers.Nadam()
opt = tfa.optimizers.Lookahead(nadam)  # lookahead 추가

InceptionResNetV2_Lookahead.compile(optimizer = opt, loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'hj/InceptionResNetV2_Lookahead.h5', verbose=1, save_best_only=True)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 10)

In [ ]:
InceptionResNetV2_Lookahead.fit_generator(train_gen, epochs=30, validation_data=val_gen, callbacks=[checkpoint, early_stop_cb])

#### 2. (Trained) 1번 모델에 30 epochs 추가하여 학습시킨 모델 저장하기
- 30 epochs + 10 epochs
- acc : 90.04%

In [ ]:
nceptionResNetV2_Lookahead_more = load_model('hj/InceptionResNetV2_Lookahead.h5')

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'hj/InceptionResNetV2_Lookahead_more.h5', verbose=1, save_best_only=True)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 10)

In [ ]:
InceptionResNetV2_Lookahead_more.fit_generator(train_gen, epochs=30, validation_data=val_gen, callbacks=[checkpoint, early_stop_cb])

#### 2. 1번 모델(30 epochs)에 cycle scheduling을 이용해서 더 학습시키기
- 30 epochs + 12 epochs (until best model) 
- acc : 91.55%

In [ ]:
InceptionResNetV2_Lookahead_more_cycle = load_model('hj/InceptionResNetV2_Lookahead.h5')

In [ ]:
n_epochs = 30
first_decay_steps = (math.ceil(45000 // 32) * n_epochs) // 7   # 4 epoch
initial_learning_rate = 0.001

lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate,
      first_decay_steps,
      t_mul=2.0,   # 주기를 늘려갈 비율
      m_mul=0.9))  # 매 주기마다 곱해줄 값

In [ ]:
adam = optimizer = keras.optimizers.Adam(learning_rate = lr_decayed_fn)
opt = tfa.optimizers.Lookahead(adam) 

InceptionResNetV2_Lookahead_more_cycle.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = [tf.keras.metrics.BinaryAccuracy()])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'hj/InceptionResNetV2_Lookahead_more_cycle.h5', verbose=1, save_best_only=True)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 10)

In [ ]:
InceptionResNetV2_Lookahead_more_cycle.fit_generator(train_gen, epochs=30, validation_data=val_gen, callbacks=[checkpoint, early_stop_cb])

#### 3. Data를 grayscale로 generate하여 Conv layer 하나  + (Pretrained) Xception
- 30 epochs
- acc : 79%

In [ ]:
Xception = tf.keras.Sequential([tf.keras.layers.Conv2D(3, (3, 3), padding='same', input_shape=[256, 256, 1]),
                              tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3)),
                              tf.keras.layers.GlobalAveragePooling2D(),
                              tf.keras.layers.Dense(1024, kernel_initializer='he_normal'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Activation('relu'),
                              tf.keras.layers.Dense(512, kernel_initializer='he_normal'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Activation('relu'),
                              tf.keras.layers.Dense(256, kernel_initializer='he_normal'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Activation('relu'),
                              tf.keras.layers.Dense(26, kernel_initializer='he_normal', activation='sigmoid', name='predictions')
                              ])

In [ ]:
Xception.compile(optimizer = tf.keras.optimizers.Nadam(), loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'hj/Xception_2nd.h5', verbose=1, save_best_only=True)
early_stop_cb = keras.callbacks.EarlyStopping(patience = 10)

In [ ]:
Xception.fit(train_gen, epochs=50, validation_data=val_gen, callbacks=[checkpoint, early_stop_cb])

#### 4. SWA 적용
- 코드 생략

### Prediction

In [ ]:
# 데이터 준비
test_submit = pd.read_csv('dacon_m12/sample_submission.csv')
test_df = test_submit.copy()
test_df['index'] = test_submit['index'].apply(lambda x: str("{:0>5d}".format(x))+'.png')

In [ ]:
# TTA 적용하기 위해 augmentation 해주기
test_gen = ImageDataGenerator(rescale=1./255.,
                              rotation_range = 30,
                              horizontal_flip = True,
                              vertical_flip = True)

test_gen = test_gen.flow_from_dataframe(dataframe = test_df,        
                                        directory='./test_route/test_dirty_mnist_2nd',      
                                        x_col='index',                             
                                        batch_size = 32,               
                                        shuffle = False,                
                                        color_mode = "rgb",           
                                        class_mode=None,
                                        seed=1,
                                        target_size=(256, 256))

In [ ]:
batch_size = 32
tta_steps = 30
predictions = []
columns = list(test_df.columns[1:])
filename = "submit.csv"

for i in tqdm(range(tta_steps)):
    print("iter {}".format(i))
    preds = model.predict_generator(generator = test_gen, steps = 5000 // batch_size, verbose = 1)
    predictions.append(preds)

# 평균을 통한 final prediction
pred = np.mean(predictions, axis=0)
pred_sub = pred.copy()

# get label
pred_sub = pred_sub.round()

# create DF
res = pd.DataFrame(pred_sub, columns = columns )
int_rest = res.astype(int)
submit = pd.concat([test_submit.iloc[:,0],int_rest], axis = 1)

submit.to_csv(filename, index = False)